# Freeze pretrained model

In [1]:
from keras.applications import VGG16
conv_base=VGG16(weights="imagenet",
               include_top=False,
               input_shape=(150,150,3))

Using TensorFlow backend.


In [2]:
from keras import layers
from keras import models

model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# Freeze conv_base's params

In [3]:
print("Before freezing, trainable params num: "+str(len(model.trainable_weights)))
conv_base.trainable=False
print("After freezing, trainable params num: "+str(len(model.trainable_weights)))

Before freezing, trainable params num: 30
After freezing, trainable params num: 4


In [4]:
from keras.preprocessing.image import ImageDataGenerator
import os
train_gen=ImageDataGenerator(rescale=1/255.,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode="nearest")
test_gen=ImageDataGenerator(rescale=1/255.)

def getGen(gen,dir_):
    return gen.flow_from_directory(dir_,
                                  target_size=(150,150),
                                  batch_size=20,
                                  class_mode="binary")
train_dir=os.path.join("small_train","train")
val_dir=os.path.join("small_train","val")
train_gener=getGen(train_gen,train_dir)
val_gener=getGen(test_gen,val_dir)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Attention! We just used VGG16 as the pretrained model, so make the learning rate small, like e-5.

In [9]:
from keras import optimizers
model.compile(
            #optimizer="rmsprop",
             optimizer=optimizers.RMSprop(lr=2e-5),
             loss="binary_crossentropy",
             metrics=["acc"])
history=model.fit_generator(train_gener,
                            steps_per_epoch=100,
                            epochs=30,
                            validation_data=val_gener,
                            validation_steps=50)

Epoch 1/30
100/100 [==============================] - 27s 270ms/step - loss: 0.3565 - acc: 0.8390 - val_loss: 0.2951 - val_acc: 0.8780
Epoch 2/30
100/100 [==============================] - 27s 266ms/step - loss: 0.3573 - acc: 0.8315 - val_loss: 0.2963 - val_acc: 0.8820
Epoch 3/30
100/100 [==============================] - 27s 266ms/step - loss: 0.3580 - acc: 0.8445 - val_loss: 0.2920 - val_acc: 0.8790
Epoch 4/30
100/100 [==============================] - 27s 267ms/step - loss: 0.3585 - acc: 0.8380 - val_loss: 0.2906 - val_acc: 0.8800
Epoch 5/30
100/100 [==============================] - 27s 265ms/step - loss: 0.3621 - acc: 0.8315 - val_loss: 0.2902 - val_acc: 0.8830
Epoch 6/30
100/100 [==============================] - 26s 262ms/step - loss: 0.3486 - acc: 0.8460 - val_loss: 0.2903 - val_acc: 0.8830
Epoch 7/30
100/100 [==============================] - 27s 267ms/step - loss: 0.3493 - acc: 0.8415 - val_loss: 0.2889 - val_acc: 0.8820
Epoch 8/30
100/100 [==============================] - 2

In [11]:
test_dir=os.path.join("small_train","test")
test_gener=getGen(test_gen,test_dir)
loss,acc=model.evaluate_generator(test_gener)

print("Test loss: "+str(loss)+". Test acc: "+str(acc))

Found 1000 images belonging to 2 classes.
Test loss: 0.262577472031. Test acc: 0.893999993801
